<a href="https://colab.research.google.com/github/marbortoli/GoogleColab/blob/main/TFOD_MASKRCNN_Marina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pathlib


In [ ]:
# Clone tensorflow models repository
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

## import dataset with images and annotations (in train and test folders)

In [ ]:
# import dataset with images and annotations (in train and test folders)

In [9]:
from google.colab import drive
drive.mount('/content/gdrive')

!ls gdrive/MyDrive/dataset


#dataset = "./gdrive/My Drive/dataset"
#!wget -q --show-progress 'https://drive.google.com/drive/folders/1ZGzVo2zeDqmSjyY2BbfBMJebELQAU_FN?usp=sharing'

Mounted at /content/gdrive


In [ ]:
# v7 database
!pip install darwin-py

In [ ]:
# authenticate
import darwin

!darwin authenticate
# key yjoOEYG.Q9JUmWG2o7jfqQ-SV3eLlRnzrO01Y7Yr

In [ ]:
# import images from v7
!darwin dataset pull energy-machines-cloud-object-detection/scada-hmi_01:exportdata0145

In [ ]:
!python create_coco_tf_record.py --logtostderr --train_image_dir=microcontroller-segmentation/train --test_image_dir=microcontroller-segmentation/test --train_annotations_file=microcontroller-segmentation/train.json --test_annotations_file=microcontroller-segmentation/test.json --output_dir=./


In [ ]:
# run tf_records script for JSON annotation data

!python /content/models/research/object_detection/dataset_tools/create_coco_tf_record.py  --logtostderr --train_image_dir = dataset/images/train --test_image_dir = dataset/images/test --train_annotations_file = dataset/annotations/train --test_annotations_file = dataset/annotations/test --output_dir=./

In [ ]:
train_record_path = '/content/train.record'
test_record_path = '/content/test.record'
labelmap_path = '/content/labelmap.pbtxt'

In [1]:
# get ready labelmap
!wget https://github.com/marbortoli/GoogleColab/blob/main/scada_label_map.pbtxt

--2022-04-21 13:58:42--  https://github.com/marbortoli/GoogleColab/blob/main/scada_label_map.pbtxt
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘scada_label_map.pbtxt’

scada_label_map.pbt     [ <=>                ] 145.61K  --.-KB/s    in 0.05s   

2022-04-21 13:58:42 (2.88 MB/s) - ‘scada_label_map.pbtxt’ saved [149102]



## configure training

In [ ]:
# model basic parameters configuration

batch_size = 4
num_steps = 3000
num_eval_steps = 1000

In [ ]:
# select pretrained model from tensorflow model zoo
#  Tensorflow 2 Detection Model Zoo
## I want mask_rcnn to make use of the json quality

!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.tar.gz
!tar -xf mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.tar.gz

In [ ]:
fine_tune_checkpoint = 'mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/ckpt-0'

In [ ]:
!wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.config

base_config_path = 'mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.config'

In [ ]:
# edit configuration file

import re

with open(base_config_path) as f:
    config = f.read()

with open('model_config.config', 'w') as f:
  
  # Set labelmap path
  config = re.sub('label_map_path: ".*?"', 
             'label_map_path: "{}"'.format(labelmap_path), config)
  
  # Set fine_tune_checkpoint path
  config = re.sub('fine_tune_checkpoint: ".*?"',
                  'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), config)
  
  # Set train tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 
                  'input_path: "{}"'.format(train_record_path), config)
  
  # Set test tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 
                  'input_path: "{}"'.format(test_record_path), config)
  
  # Set number of classes.
  config = re.sub('num_classes: [0-9]+',
                  'num_classes: {}'.format(4), config)
  
  # Set batch size
  config = re.sub('batch_size: [0-9]+',
                  'batch_size: {}'.format(batch_size), config)
  
  # Set training steps
  config = re.sub('num_steps: [0-9]+',
                  'num_steps: {}'.format(num_steps), config)
  
  # Set fine-tune checkpoint type to detection
  config = re.sub('fine_tune_checkpoint_type: "classification"', 
             'fine_tune_checkpoint_type: "{}"'.format('detection'), config)
  
  f.write(config)

In [ ]:
%cat model_config.config

In [ ]:
model_dir = 'training/'
pipeline_config_path = 'model_config.config'

## train custom model

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_config_path} \
    --model_dir={model_dir} \
    --alsologtostderr

In [ ]:
# visualize model efficiency 

%load_ext tensorboard
%tensorboard --logdir '/content/training/train'

## export model inference graph

In [ ]:
output_directory = 'inference_graph'

!python /content/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir {model_dir} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_config_path}

In [ ]:
# download

from google.colab import files
files.download(f'/content/{output_directory}/saved_model/saved_model.pb') 